<a href="https://colab.research.google.com/github/OmarMohammed88/NLP_Tasks/blob/main/Task_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn import metrics
from pprint import pprint
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os, sys
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from nltk.corpus import names
import nltk
nltk.download('names')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.datasets import fetch_rcv1


[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import numpy as np


In [ ]:
%tensorflow_version 1.x


TensorFlow 1.x selected.


In [ ]:
!pip install bert-serving-client

In [ ]:
!pip install -U bert-serving-server[http]

Requirement already up-to-date: bert-serving-server[http] in /usr/local/lib/python3.6/dist-packages (1.10.0)


In [ ]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip


--2020-08-28 22:32:33--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.200.128, 74.125.68.128, 172.253.118.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.200.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   124MB/s    in 3.1s    

2020-08-28 22:32:37 (124 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]



In [ ]:
!unzip '/content/uncased_L-12_H-768_A-12.zip'

Archive:  /content/uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [ ]:
!nohup bert-serving-start -model_dir=/content/uncased_L-12_H-768_A-12 > out.file 2>&1 &

In [ ]:
from bert_serving.client import BertClient


In [ ]:
bc = BertClient()

In [ ]:
nltk.download('reuters')
nltk.download('punkt')
from nltk.corpus import stopwords, reuters

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
documents = reuters.fileids()
 
train_docs_id = list(filter(lambda doc: doc.startswith("train"),
                            documents))
test_docs_id = list(filter(lambda doc: doc.startswith("test"),
                           documents))
 
train_docs = [reuters.raw(doc_id) for doc_id in train_docs_id]
test_docs = [reuters.raw(doc_id) for doc_id in test_docs_id]

In [ ]:
docs_pdf_train = pd.DataFrame(train_docs) 
docs_pdf_test = pd.DataFrame(test_docs) 
docs_pdf_train['text'] = pd.DataFrame(train_docs) 
docs_pdf_test['text'] = pd.DataFrame(test_docs) 
docs_pdf_train.drop([docs_pdf_train.columns[0]], axis ='columns',inplace=True)
docs_pdf_test.drop([docs_pdf_test.columns[0]], axis ='columns',inplace=True)
docs_pdf_train.dropna(inplace=True)
docs_pdf_test.dropna(inplace=True)

In [ ]:
docs_pdf_train.head()

,text
0,BAHIA COCOA REVIEW\n Showers continued throug...
1,COMPUTER TERMINAL SYSTEMS &lt;CPML> COMPLETES ...
2,N.Z. TRADING BANK DEPOSIT GROWTH RISES SLIGHTL...
3,NATIONAL AMUSEMENTS AGAIN UPS VIACOM &lt;VIA> ...
4,ROGERS &lt;ROG> SEES 1ST QTR NET UP SIGNIFICAN...


In [ ]:
docs_pdf_train.dropna(subset = ["text"], inplace=True)
docs_pdf_train.replace(r'-', ' ', regex=True,inplace=True)
docs_pdf_train.replace(r'\t', ' ', regex=True,inplace=True)
docs_pdf_train.replace(r'\n', ' ', regex=True,inplace=True)
docs_pdf_train.replace(r'   ', ' ', regex=True,inplace=True)
docs_pdf_test.replace(r'-', ' ', regex=True,inplace=True)
docs_pdf_test.replace(r'\t', ' ', regex=True,inplace=True)
docs_pdf_test.replace(r'\n', ' ', regex=True,inplace=True)
docs_pdf_test.replace(r'   ', ' ', regex=True,inplace=True)

In [ ]:
all_names = names.words()
WNL = WordNetLemmatizer()
def clean(data):
    cleaned = defaultdict(list)
    count = 0
    for group in data:
        for words in group.split():
            if words.isalpha() and words not in all_names:
                cleaned[count].append(WNL.lemmatize(words.lower()))
        cleaned[count] = ' '.join(cleaned[count])
        count +=1 
    return(list(cleaned.values()))
docs_pdf_train['text'] = clean(docs_pdf_train['text'])
docs_pdf_test['text'] = clean(docs_pdf_test['text'])

KeyboardInterrupt: ignored

In [ ]:
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import re
 

In [ ]:
cachedStopWords = stopwords.words("english")
 
def tokenize(text):
  min_length = 3
  words = map(lambda word: word.lower(), word_tokenize(text))
  words = [word for word in words if word not in cachedStopWords]
  tokens = (list(map(lambda token: PorterStemmer().stem(token),
                                   words)))
  p = re.compile('[a-zA-Z]+');
  filtered_tokens =list(filter (lambda token: p.match(token) and
                               len(token) >= min_length,
                               tokens))
  return filtered_tokens

In [ ]:
def word_tokenizer():
  tokenize_docs = []
  docs_tokens = docs_pdf_train['text'].tolist()
  length = len(docs_tokens)
  for i in range(length):
    tokenize_docs.append(tokenize(docs_tokens[i]))
  return tokenize_docs

In [ ]:
tokenize_docs = word_tokenizer()


In [ ]:
# sentences_embeddings_retuers_train=bc.encode(docs_pdf_train['text'])
sentences_embeddings_retuers_train=[]
for i in range(len(tokenize_docs)):
  vector=bc.encode(tokenize_docs[i])
  sentences_embeddings_retuers_train.append(vector)

In [ ]:
from numpy import save
data = np.asarray(sentences_embeddings_retuers_train)
save('data.npy', data)

In [ ]:
def word_tokenizer_1():
  tokenize_docs = []
  docs_tokens = docs_pdf_test['text'].tolist()
  length = len(docs_tokens)
  for i in range(length):
    tokenize_docs.append(tokenize(docs_tokens[i]))
  return tokenize_docs
tokenize_docs_test=word_tokenizer_1()

In [ ]:
sentences_embeddings_retuers_test=[]
for i in range(len(tokenize_docs_test)):
  vector=bc.encode(tokenize_docs_test[i])
  sentences_embeddings_retuers_test.append(vector)

In [ ]:
data_set= np.asarray(sentences_embeddings_retuers_test)
save('data_test.npy', data_set)

In [ ]:
sentences_embeddings_retuers_test

## Classification


In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MultiLabelBinarizer


In [ ]:
mlb = MultiLabelBinarizer()

In [ ]:
train_labels = mlb.fit_transform([reuters.categories(doc_id)for doc_id in train_docs_id])

In [ ]:
test_labels = mlb.fit_transform([reuters.categories(doc_id)for doc_id in test_docs_id])

In [ ]:
classifier = OneVsRestClassifier(SGDClassifier(max_iter=50, tol=1e-3))

In [ ]:
classifier.fit(final_train_list, train_labels)

In [ ]:
!cp '/content/drive/My Drive/data.npy' '/content/'
!cp '/content/drive/My Drive/data_test.npy' '/content/'


In [ ]:
from numpy import load
# load array
sentences_embeddings_retuers_train = load('data.npy',allow_pickle=True)


In [ ]:
sentences_embeddings_retuers_test = load('data_test.npy',allow_pickle=True)


In [ ]:
sentences_embeddings_retuers_train[0].shape

(218, 768)

In [ ]:
listOfList = []
flatList = []
flatList2=[]

In [ ]:
#not used anymore
for i in range(len(sentences_embeddings_retuers_train)):
  a7a=sentences_embeddings_retuers_train[i]
  for elem in a7a:
    flatList.extend(elem)
  listOfList.append(flatList)
  flatList=[]
  


In [ ]:
from numpy import save
import numpy as np
# listOfList_np= np.asarray(listOfList)
# save('listOfList_np_2.npy', listOfList_np)

In [ ]:
#Flatten USED
import itertools
for i in range(3884):
  y=sentences_embeddings_retuers_train[i]
  x=[i for i in itertools.chain.from_iterable(y)]
  flatList.append(x)

In [ ]:
flat_list1=np.asarray(flatList)
# save('listOfList_np_1.npy', flat_list1)

In [ ]:
flat_list2=np.asarray(flatList2)


In [ ]:
def pre_allocate(length,arr1,arr2,arr3,arr4,arr5,arr6,arr7,arr8,arr9,arr10,arr11,arr12):
    counter=0
    results = np.empty(length)
    for i in range(len(arr1)):
      results[i]=arr1[i]
      counter+=1
    for i in range(counter,len(arr2)):
      results[counter]=arr2[i]
      counter+=1

    for i in range(len(arr3)):
      results[counter]=arr3[i]
      counter+=1

    for i in range(len(arr4)):
      results[counter]=arr4[i]
      counter+=1

    for i in range(len(arr5)):
      results[counter]=arr5[i]
      counter+=1

    for i in range(len(arr6)):
      results[counter]=arr6[i]
      counter+=1

    for i in range(len(arr7)):
      results[counter]=arr7[i]
      counter+=1

    for i in range(len(arr8)):
      results[counter]=arr8[i]
      counter+=1

    for i in range(len(arr9)):
      results[counter]=arr9[i]
      counter+=1

    for i in range(len(arr10)):
      results[counter]=arr10[i]
      counter+=1

    for i in range(len(arr11)):
      results[counter]=arr11[i]
      counter+=1

    for i in range(len(arr12)):
      results[counter]=arr12[i]
      counter+=1
        
    return results

In [ ]:
def numpy_append():
    results = np.array([])
    for i in range(length):
        np.append(results, )
    return results

In [ ]:
li=[[1,2],[1,2],[1,2],[1,2]]


ValueError: ignored